In [1]:
%load_ext autoreload
%matplotlib inline

import os
import sys

print(os.getcwd())
os.chdir(os.path.expanduser('~') + '/Code/vail')
print(os.getcwd())

/Users/msc/Code/vail/ml/notebooks
/Users/msc/Code/vail


In [2]:
%autoreload 2
import ml

In [97]:
# o = ml.Options({'environment': 'walker2d-medium-expert-v2'})
# o = ml.Options({'environment': 'hopper-medium-expert-v2'})
o = ml.Options({'environment': 'halfcheetah-medium-expert-v2'})
d = ml.OfflineDataset(o)
d._build()

load datafile: 100%|██████████| 9/9 [00:05<00:00,  1.52it/s]


Output()

Output()

In [27]:
import matplotlib.pyplot as plt
import gym
import numpy as np
import torch
import d4rl
import random

from torchvision.transforms.functional import crop, resize
from einops import rearrange

from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib as mpl
import numpy as np

import imageio

mpl.use('qtagg')


In [99]:
# env = gym.make('Walker2d-v2')
# env = gym.make('Hopper-v2')
env = gym.make('HalfCheetah-v2')

In [105]:
render_size = 256
size = 32

n = 10
frames = []

def get_image():

    rand = random.randint(0, len(d.QP))
    qp, qv = d.QP[rand], d.QV[rand]
    env.set_state(qp, qv)  # type: ignore

    with ml.RedirectStream():
        frame = env.sim.render(render_size, render_size, camera_name='track', mode='offscreen')  # type: ignore
    frame = np.flip(frame, axis=0)
    frame = torch.from_numpy(frame.copy())
    frame = frame.to(torch.uint8)
    frame = frame.permute(2, 0, 1)

    # frame = crop(frame, top=64, left=64, width=128, height=192)
    # frame = resize(frame, (size, size), antialias=True)

    return frame
    
frame = get_image()
print(frame.shape)
# plt.imshow(frame.permute(1, 2, 0))
# plt.show()

imageio.imwrite('still.png', frame.permute(1, 2, 0))

torch.Size([3, 256, 256])


In [ ]:

patch_size = 32
img = get_image()

img = rearrange(img, 'c (h ph) (w pw) -> h w ph pw c', ph=patch_size, pw=patch_size)

print(img.shape)
h, w, _, _, c = img.shape

fig = plt.figure(figsize=(4., 4.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(h, w),  # creates 2x2 grid of axes
                 axes_pad=0.05,  # pad between axes in inch.
                 )

for i in range(h):
    for j in range(w):
        g = grid.axes_row[i][j]
        g.imshow(img[i, j])
        g.axis('off')

plt.show()